In [ ]:
%load_ext autoreload
%autoreload 2

from os.path import join as pjoin
import pandas as pd
from datetime import datetime
from bmp_config import path_data
from bmp_behav_proc import *
import seaborn as sns

fnf = pjoin(path_data,'df_all_multi_tsz__.pkl.zip')
print(fnf)
print( str(datetime.fromtimestamp(os.stat(fnf).st_mtime)))
df_all_multi_tsz = pd.read_pickle(fnf)
df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe" '
                           ' and retention_factor_s == "0.924"').copy().sort_values(['subject','trials'])
df,dfall,ES_thr,envv,pert = addBehavCols2(df);
assert df.groupby('subject', observed=True).size().min() == 768
df_wthr = df.copy()

# Error consist vs ES (Fig 5, Herzfeld test)

## Error change

In [ ]:
dfc = df_wthr.copy() # df_thr is untruncated and we need it to compute error change
dfc = dfc.query('err_sens.abs() <= @ES_thr')

dfcs_ = truncateDf(dfc, 'err_sens_change', 
       q=0.0, infnan_handling='discard', 
       cols_uniqify = ['subject','env'], verbose=True)

# make sure I have not changed the algo
assert len(dfcs_.query('err_sign_same == 0')) == 0

In [ ]:
getQueryPct(dfcs_,'err_sign_same == 0')
for i in range(2,4):
    getQueryPct(dfcs_,f'err_sign_same{i} == 0')

In [ ]:
# Error change
import warnings
from figure.imgfilemanip import *
from figure.mystatann import plotSig0All, plotSig0, plotSigAll
from figure import renameTickLabels
dfcs2  = truncateDf(dfcs_, 'err_sens_change', q=0.0,
                    infnan_handling='discard', 
                   cols_uniqify = ['subject','env'])

#coln_same = 'err_sign_same3'
coln_same = 'err_sign_same'

# add a new column that is a concatenation of coln_same column and env column
coln_ = coln_same + '_env'
dfcs2[coln_] = dfcs2[coln_same].astype(str) + '_' + dfcs2['env'].astype(str)

qs = f'{coln_same} != 0'
#grp = dfcs2.query(qs).groupby(['subject',coln_same,'env'], observed=True)
grp = dfcs2.query(qs).groupby(['subject',coln_,'env'], observed=True)
dfme = grp.mean(numeric_only=1).reset_index()
#dfme['Environment'] = dfme['env']

coln_vals = ['-1_stable','-1_random','1_stable','1_random']
with warnings.catch_warnings():
    warnings.filterwarnings('ignore',category=FutureWarning)
    warnings.filterwarnings('ignore',category=UserWarning)
    # fg = sns.catplot(data=dfme, kind='violin',
    #      y ='err_sens_change', x=coln_same,
    #      hue='Environment',hue_order=['stable','random'],
    #      palette=['tab:orange','tab:grey'],
    #     legend_out=False) #showfliers=True, 
    fg = sns.catplot(data=dfme, kind='violin',
         y ='err_sens_change', x=coln_,
         order=coln_vals, 
         hue=coln_,hue_order=coln_vals,
         palette=['tab:orange','tab:grey','tab:orange','tab:grey'],
        legend_out=False) #showfliers=True, 
#fg.apply(lambda ax: ax.axline(y=0))
#def f(data, **kwargs):
#    plt.axhline(y=0)
#fg.map(f, ls=":", c=".5" )
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',c='r')
    
pairs = [('1_random', '-1_random'), ('1_stable', '-1_stable'),
         ('-1_stable', '-1_random'), ('1_stable', '1_random'), ]
plotSigAll(ax, 0.81, 0.18, ticklen=0.05,  df=dfme, colpair=coln_, coln='err_sens_change',
    pairs = pairs)#, lab2tick = )

#for xv in [-1,1]:
# ttrs0,ttrssigs0 = [],[]
# for env_,xt,xv in [('stable',-0.25,-1),('random',0.25,-1),
#                   ('stable',0.75,1),('random',1., 1)]:
#     for alt in ['greater','less','two-sided']:
#         ttrs0_,ttrssig0 = plotSig0(ax, x=xv, y=1.46, df=dfme.query('env == @env_'), 
#                  coln='err_sens_change', colx = coln_same, xt=xt, alt=alt)
#         ttrs0_['env'] = env_
#         ttrs0_[coln_same] = xv
#         ttrs0 += [ttrs0_]
#         if len(ttrssig0):
#             row = ttrssig0.iloc[0]
#             ttrssig0['env'] = env_
#             ttrssig0[coln_same] = xv
#             #print(env_,xv,row['pval'],ttstr)
#             ttrssigs0 += [ttrssig0]
# ttrssigs0 = pd.concat(ttrssigs0, ignore_index=1)
# ttrs0 = pd.concat(ttrs0, ignore_index=1)
plotSig0All(ax, 1.56, df=dfme, colpair=coln_, coln='err_sens_change', show_ns = 1)

plt.suptitle('Error sensitivity dependence \non error sign consistency',y=1)
ax.get_figure().subplots_adjust(top=0.85)

#ax.set_ylabel('Error sensitivity change,\nmean within participant')
ax.set_ylabel('Error sensitivity change')
ax.set_xlabel('Sign consistency')
renameTickLabels(ax, dict(zip(coln_vals, ['stable\ninconsistent', 'random\ninconsistent',
                              'stable\nconsistent', 'random\nconsistent'])))
fnffigbase = pjoin(path_fig, 'behav', f'ESchange_per_sign_consist_{coln_same}')
plt.savefig(fnffigbase + '.svg')
plt.savefig(fnffigbase + '.pdf')
plt.close()
svg2png(fnffigbase + '.svg', dpi=300)

from IPython.display import Image
display(Image(fnffigbase + '_dpi=300.png'))

In [ ]:
dfcs2  = truncateDf(dfcs_.query('trialwpertstage_wb <= 60'), 'err_sens_change', q=0.0,
                    infnan_handling='discard', 
                   cols_uniqify = ['subject','env'])
fg = sns.relplot(data=dfcs2, kind='line',  x='trialwpertstage_wb',y='err_sens_change', hue='err_sign_same', col='env' )
fg.refline(y=0)
plt.show()

In [ ]:
del dfme

# Stats

In [ ]:
# Error change, but without trials right after pert (first time when participant
# could realize something is wrong and start to correct in the opposit direction)
dfcs2  = truncateDf(dfcs_, 'err_sens_change', q=0.0,
                    infnan_handling='discard', 
                   cols_uniqify = ['subject','env']).query('trialwpertstage_wb != 1')

#coln_same = 'err_sign_same3'
coln_same = 'err_sign_same'
qs = f'{coln_same} != 0.'
grp = dfcs2.query(qs).groupby(['subject',coln_same,'env'],
                             observed=True)
dfme = grp.mean(numeric_only=1).reset_index()
dfme['Environment'] = dfme['env']

with warnings.catch_warnings():
    warnings.filterwarnings('ignore',category=FutureWarning)
    warnings.filterwarnings('ignore',category=UserWarning)
    fg = sns.catplot(data=dfme, kind='box',
         y ='err_sens_change', x=coln_same,
         hue='Environment',hue_order=['stable','random'],
         palette=['tab:orange','tab:grey'],
        showfliers=True, legend_out=False)
#fg.apply(lambda ax: ax.axline(y=0))
#def f(data, **kwargs):
#    plt.axhline(y=0)
#fg.map(f, ls=":", c=".5" )
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',c='r')
    
ttrs0,ttrssigs0 = [],[]
for env_,xt,xv in [('stable',-0.25,-1),('random',0.25,-1),
                  ('stable',0.75,1),('random',1., 1)]:
    #for xv in [-1,1]:
    for alt in ['greater','less','two-sided']:
        ttrs0_,ttrssig0 = plotSig0(ax, x=xv, y=0.63, df=dfme.query('env == @env_'), 
                 coln='err_sens_change', colx = coln_same, xt=xt, alt=alt)
        ttrs0_['env'] = env_
        ttrs0_[coln_same] = xv
        ttrs0 += [ttrs0_]
        if len(ttrssig0):
            row = ttrssig0.iloc[0]
            ttrssig0['env'] = env_
            ttrssig0[coln_same] = xv
            #print(env_,xv,row['pval'],ttstr)
            ttrssigs0 += [ttrssig0]
ttrssigs0 = pd.concat(ttrssigs0, ignore_index=1)
ttrs0 = pd.concat(ttrs0, ignore_index=1)
        
#plotSigAll(ax, 1, 0.1,  df=dfme, lab2tick = )
    
plt.suptitle('Error sensitivity dependence \non error sign consistency',y=1.1)
ax.set_ylabel('Error sensitivity change,\nmean within participant')
ax.set_xlabel('Sign consistency')
renameTickLabels(ax, {'-1':'inconsistent', '1':'consistent' })

# NFTAPO =not first trial after perturbation one
plt.savefig(pjoin(path_fig, f'ESchange_per_sign_consist_{coln_same}_NFTAPO.svg')) 
plt.savefig(pjoin(path_fig, f'ESchange_per_sign_consist_{coln_same}_NFTAPO.pdf')) 

In [ ]:
# calc stats
# for positive, negative signs of 
display(ttrssigs0)
s0 =''
for i,row in ttrssigs0.query('alt != "two-sided"').iterrows():
    sign = row['ttstr'][-3:]
    sc = None
    if row[coln_same] == 1:
        sc = 'consistent'
    elif row[coln_same] == -1:
        sc = 'inconsistent'
    #s = 'In {} environment for {} consecutive error signs ES change mean is {:.2f} (std={:.2f}) {} with p-value = {:.2e}'.\
    #    format(row['env'],sc,row['err_sens_change_mean'],row['err_sens_change_std'],sign,row['pval'])
    s = 'In {} environment for {} consecutive error signs ES change = '.format(row['env'],sc) + report_ttest(row,'err_sens_change')
    s0 += s + '.\n'
print(s0)

s0 =''
for i,row in ttrs0.query('alt == "two-sided" and pval > 0.05').iterrows():
    sign = row['ttstr'][-3:]
    sc = None
    if row[coln_same] == 1:
        sc = 'consistent'
    elif row[coln_same] == -1:
        sc = 'inconsistent'
    #s = 'In {} environment for {} consecutive error signs ES change mean {:.2f} has unclear sign (std={:.2f}) with p-value = {:.2e}'.\
    #    format(row['env'],sc,row['err_sens_change_mean'],row['err_sens_change_std'],row['pval'])
    s = 'In {} environment for {} consecutive error signs ES change = '.format(row['env'],sc) + report_ttest(row, 'err_sens_change')
    s0 += s + '.\n'
print(s0)

In [ ]:
#%debug
from figure.mystatann import square_updiag
from itertools import product
def f(df, coln, colx, colhue, colx_vals= None, colhue_vals=None,
     xts=None):
    
    if colx_vals is None:
        colx_vals = list(sorted(df[colx].unique()))
    if colhue_vals is None:
        colhue_vals = list(sorted(df[colhue].unique()))
    hierpairs = product(colx_vals, colhue_vals)
    
    qss = []
    for a,b in hierpairs:
        if isinstance(b,str):
            b = '"' + b + '"'
        if isinstance(a,str):
            a = '"' + a + '"'
        qs = '{} == {} and {} == {}'.format(colx,a,colhue,b)
        qss += [qs]
        
    #print(qss)
    qs_pairs = square_updiag(qss)
    ttrssig,ttrs = comparePairs(df, coln, [colx,colhue], paired=True, qspairs = qs_pairs)
    return ttrssig,ttrs
ttrssig,_ = f(dfme,'err_sens_change', coln_same, 'env')
ttrssig

In [ ]:
import re    
def parseQs(qs):
    pattern = r"(?P<var1>\w+)\s*==\s*(?P<val1>.*?)\s+and\s+(?P<var2>\w+)\s*==\s*(?P<val2>.*)"
    match = re.search(pattern, qs)
    if match:
        var1 = match.group('var1')
        val1 = match.group('val1')
        var2 = match.group('var2')
        val2 = match.group('val2')
        #print(f"Variable 1: {var1}, Value 1: {val1}")
        #print(f"Variable 2: {var2}, Value 2: {val2}")
        
        return {var1:eval(val1), var2:eval(val2 )}
    print('no match for ',qs)
    return None
#parseQs(s)
# for s in set(ttrssig.qs1.unique()) | set(ttrssig.qs2.unique()) :    
#     print(s)
#     d = parseQs(s)
#     print(d)

sign2nice = {1:'consistent', -1:'inconsistent'}

#ttrssig_ps,_ = comparePairs(me_ps, 'err_sens', 'ps_', alt=['greater'], paired=True, updiag=False)
#display(ttrssig_ps[ttrssig_ps.columns[-5:]])
# within env
s1 = ''
for i,row in ttrssig.query('alternative != "two-sided"').iterrows():
    
    d1 = parseQs( row['qs1'] )
    d2 = parseQs( row['qs2'] )
    sc1 = sign2nice[d1[coln_same]]
    sc2 = sign2nice[d2[coln_same]]
    
    if d1['env'] != d2['env']:
        continue
    s = f"ES change for {d1['env']} environment for {sc1} errors is {row['alternative']} than "
    s += f"ES change for {d2['env']} environment for {sc2} errors is {row['alternative']},"
    #ttstr = row['ttstr']
    #s = ttstr.replace(row['val2'], ps_2nice[row['val2']] ).replace(row['val1'], ps_2nice[row['val1']] )    
    s += f" p-value={row['pval']:.2e}; \n"
    s1 += s
#s = f"Stable > random p-value={row['pval']:.2e}"
print(s1)